In [1]:
import pandas as pd

In [25]:
df1 = pd.read_csv(r'E:\ML Learning\PROJECT_SENTIMENT_ANALYSIS_WHITE_HOUSE\STEP_4_CORRELATION\data_files\roberta_scored_briefings.csv')
df2 = pd.read_csv(r'E:\ML Learning\PROJECT_SENTIMENT_ANALYSIS_WHITE_HOUSE\STEP_4_CORRELATION\data_files\approvalRating.csv')

In [26]:
df2

,approving,disapproving,end_date,start_date,unsure
0,38,58,6/23/2024,06-03-24,4
1,39,56,5/23/2024,05-01-24,5
2,38,58,4/22/2024,04-01-24,4
3,40,55,3/20/2024,03-01-24,5
4,38,59,2/20/2024,02-01-24,3
5,41,54,1/22/2024,01-02-24,5
6,39,59,12/20/2023,12/1/2023,2
7,37,59,11/20/2023,11-01-23,4
8,37,59,10/23/2023,10-02-23,4
9,41,58,9/23/2023,09-01-23,1
